# Leave-One-Subject-Out Evaluation

In [1]:
import numpy as np
import cv2
import dlib
from imutils import face_utils
import glob
import pickle
from random import shuffle
from sklearn.model_selection import train_test_split
from sklearn import metrics
import json
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

### 3D Model points for SolvePnP
We will approximate the 3D points of the following face parts with the correspoinding coordinates. This is a general model of the human face and we do not need to worry much about absolute accuracy.

In [2]:
# 3D model points.
model_points = np.array([
                            (0.0, 0.0, 0.0),             # Nose tip
                            (0.0, -330.0, -65.0),        # Chin
                            (-225.0, 170.0, -135.0),     # Left eye left corner
                            (225.0, 170.0, -135.0),      # Right eye right corne
                            (-150.0, -150.0, -125.0),    # Left Mouth corner
                            (150.0, -150.0, -125.0)      # Right mouth corner
                         
                        ])

### Load Dataset

In [107]:
# Number of training examples to use(0-2806)
DATASET_SIZE = 2758
DEBUG = False

# Load the dataset
with open('data_cleaned.json') as json_file:
    data_all = json.load(json_file)

# Extract the keys in sorted order
keys = sorted(data_all)

# Convert python list to np array
keys = np.asarray(keys)

random_key = np.random.randint(DATASET_SIZE, size=1)
# Extract the UUID of the randomly chosen key and leave out every key with that UUID
uuid_excluded = keys[random_key][0].split('/')[0]

indices_excluded = []
keys_excluded = []
for i in range(DATASET_SIZE):
    key = keys[i]
    uuid = key.split('/')[0]
    if(uuid == uuid_excluded):
        indices_excluded.append(i)
        keys_excluded.append(key)
keys = np.delete(keys, indices_excluded)
print(keys.shape)
DATASET_SIZE = keys.shape[0]

(2695,)


In [108]:
print(uuid_excluded)

05d77af9b4ce4de5b0f631f685601d41


In [123]:
X = np.zeros((DATASET_SIZE, 6, 1))
y = np.zeros(DATASET_SIZE)

# Indices that the SolvePnP failed
failed_indices = []

for i in range(DATASET_SIZE):
    key = keys[i]

    # Approximate camera intrinsic parameters
    im = cv2.imread('dataset/' + key)   # This imread is time consuming! Another way?
    size = im.shape
    focal_length = size[1]
    center = (size[1]/2, size[0]/2)
    camera_matrix = np.array(
                             [[focal_length, 0, center[0]],
                             [0, focal_length, center[1]],
                             [0, 0, 1]], dtype = "double"
                             )
    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
    
    landmarks = data_all[key]['landmarks']
    
    # Grab the 2D coordinates of our six sample points
    image_points = np.array([
        (landmarks[33]['x'], landmarks[33]['y']) ,     # Nose tip
        (landmarks[8]['x'], landmarks[8]['y']),     # Chin
        (landmarks[36]['x'], landmarks[36]['y']),     # Left eye left corner
        (landmarks[45]['x'], landmarks[45]['y']),     # Right eye right corner
        (landmarks[48]['x'], landmarks[48]['y']),     # Left Mouth corner
        (landmarks[54]['x'], landmarks[54]['y'])      # Right mouth corner
    ], dtype="double")
    
    # Solve the PnP problem with the parameters specified above
    # and obtain rotation and translation vectors
    (success, rotation_vector, translation_vector) = cv2.solvePnP(
        model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE
        )
    
    X[i, :] = np.concatenate((rotation_vector, translation_vector), axis=0)
    # Check if it is positive or negative example
    output = key.split('/')[1]
    if(output == 'positive'):
        y[i] = 1
    elif(output == 'negative'):
        y[i] = 0
    
    # DEBUGGING
    if(X[i, 0] > 100000):
        # Project a 3D point (0, 0, 1000.0) onto the image plane.
        # We use this to draw a line sticking out of the nose
        (nose_end_point2D, jacobian) = cv2.projectPoints(
            np.array([(0.0, 0.0, 500.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs
            )
        for p in image_points:
            cv2.circle(im, (int(p[0]), int(p[1])), 3, (0,0,255), -1)
        p1 = ( int(image_points[0][0]), int(image_points[0][1]) )
        p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]) )
        
        # Draw a line connecting the two points. This line must show
        # the direction out of the nose
        cv2.line(im, p1, p2, (255,0,0), 2)
        # Display image
        cv2.imshow("Output", im)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        print("Rotation Vector:\n {0}".format(rotation_vector))
        print("Translation Vector:\n {0}".format(translation_vector))
        print(key)
        failed_indices.append(i)

X = np.delete(X, failed_indices, axis=0)
y = np.delete(y, failed_indices, axis=0)
X = X.squeeze()
print(X.shape)
print(y.shape)

Rotation Vector:
 [[ 3783405.18811562]
 [-4014160.00070105]
 [26470515.88662656]]
Translation Vector:
 [[ 1.92984857e+07]
 [-1.02404510e+08]
 [-1.11258103e+09]]
8036631b52854bdda33a928371100eca/positive/8036631b52854bdda33a928371100eca_1293_886.jpg
Rotation Vector:
 [[ 2.54330113e+08]
 [ 5.34029841e+08]
 [-1.08799044e+08]]
Translation Vector:
 [[ 3.65972863e+08]
 [-4.40021703e+09]
 [-5.56045989e+10]]
8036631b52854bdda33a928371100eca/positive/8036631b52854bdda33a928371100eca_1573_672.jpg
(2693, 6)
(2693,)


### Standardization(Works better in Gaussian-line Distributions)

In [59]:
# m = X[0:500].mean(axis=0)
# std = X[0:500].std(axis=0)

# X_scaled = (X - m)/std

### Normalization

In [125]:
max_vals = np.max(X, axis=0)
min_vals = np.min(X, axis=0)
print(max_vals)
print(min_vals)
X_scaled = (X - min_vals)/(max_vals - min_vals)
print(X_scaled[0:100])

[3.53662508e+00 1.02292301e+01 2.04586526e+00 2.95562783e+03
 7.34242137e+02 7.87620899e+03]
[-3.28019931e+00 -1.52525647e+01 -1.08399717e+00 -1.95621504e+03
 -1.24769639e+03 -9.11097263e+03]
[[0.06401037 0.60332329 0.44844224 0.47856536 0.75638585 0.64862283]
 [0.06562014 0.60379737 0.44330201 0.47748204 0.76378669 0.64861359]
 [0.05821942 0.60120712 0.45860628 0.48682453 0.74427206 0.65326301]
 [0.06493258 0.6039238  0.44557271 0.47880503 0.75864194 0.64856258]
 [0.06416714 0.60347159 0.45287532 0.48181547 0.74702586 0.64930347]
 [0.06048864 0.60223547 0.4422605  0.48501553 0.74256477 0.65017643]
 [0.06460841 0.6034585  0.44910711 0.47948495 0.75336302 0.64878946]
 [0.95307111 0.59470114 0.26224418 0.47496109 0.83521722 0.64618962]
 [0.06319534 0.60283568 0.44255611 0.4846299  0.74213297 0.6499375 ]
 [0.06636764 0.60353482 0.4437236  0.47497107 0.75868585 0.64760872]
 [0.0598278  0.60292714 0.4464201  0.48581317 0.74090933 0.64991561]
 [0.06009686 0.60200711 0.44342298 0.48595401 0.7

### Train and Predict

In [154]:
 X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

In [155]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=1)
rf_classifier.fit(X_train, y_train)
y_pred_rf = rf_classifier.predict(X_test)

print('Test set accuracy for Random Forest: ', metrics.accuracy_score(y_test, y_pred_rf))

Test set accuracy for Random Forest:  0.7402597402597403


In [156]:
svm_classifier = svm.SVC(C=100, kernel='rbf', gamma='auto')
svm_classifier.fit(X_train, y_train)
y_pred_svm = svm_classifier.predict(X_test)

print('Test set accuracy for SVM: ', metrics.accuracy_score(y_test, y_pred_svm))

Test set accuracy for SVM:  0.6178107606679035


In [157]:
print(len(keys_excluded))

63


In [145]:
X_eval = np.zeros((len(keys_excluded), 6, 1))
y_eval = np.zeros(len(keys_excluded))

for i in range(len(keys_excluded)):
    key = keys_excluded[i]
        
    # Approximate camera intrinsic parameters
    im = cv2.imread('dataset/' + key)   # This imread is time consuming! Another way?
    size = im.shape
    focal_length = size[1]
    center = (size[1]/2, size[0]/2)
    camera_matrix = np.array(
                             [[focal_length, 0, center[0]],
                             [0, focal_length, center[1]],
                             [0, 0, 1]], dtype = "double"
                             )
    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion

    landmarks = data_all[key]['landmarks']
    
    # Grab the 2D coordinates of our six sample points
    image_points = np.array([
        (landmarks[33]['x'], landmarks[33]['y']) ,     # Nose tip
        (landmarks[8]['x'], landmarks[8]['y']),     # Chin
        (landmarks[36]['x'], landmarks[36]['y']),     # Left eye left corner
        (landmarks[45]['x'], landmarks[45]['y']),     # Right eye right corner
        (landmarks[48]['x'], landmarks[48]['y']),     # Left Mouth corner
        (landmarks[54]['x'], landmarks[54]['y'])      # Right mouth corner
    ], dtype="double")
    
    # Solve the PnP problem with the parameters specified above
    # and obtain rotation and translation vectors
    (success, rotation_vector, translation_vector) = cv2.solvePnP(
        model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE
        )
    
    X_eval[i, :] = np.concatenate((rotation_vector, translation_vector), axis=0)
    # Check if it is positive or negative example
    output = key.split('/')[1]
    if(output == 'positive'):
        y_eval[i] = 1
    elif(output == 'negative'):
        y_eval[i] = 0
    
    if(True and i<10):
        # Project a 3D point (0, 0, 1000.0) onto the image plane.
        # We use this to draw a line sticking out of the nose
        (nose_end_point2D, jacobian) = cv2.projectPoints(
            np.array([(0.0, 0.0, 500.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs
            )
        for p in image_points:
            cv2.circle(im, (int(p[0]), int(p[1])), 3, (0,0,255), -1)
        p1 = ( int(image_points[0][0]), int(image_points[0][1]) )
        p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]) )
        
        # Draw a line connecting the two points. This line must show
        # the direction out of the nose
        cv2.line(im, p1, p2, (255,0,0), 2)
        # Display image
        cv2.imshow(output, im)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


X_eval = X_eval.squeeze()
print(X_eval.shape, y_eval.shape)

(63, 6) (63,)


In [71]:
# m_eval = X_eval.mean(axis=0)
# std_eval = X_eval.std(axis=0)

# X_eval = (X_eval - m_eval)/std_eval

In [146]:
min_vals = np.min(X_eval, axis=0)
max_vals = np.max(X_eval, axis=0)

X_eval = (X_eval - min_vals)/(max_vals - min_vals)

In [160]:
print(X_eval)

[[0.04196262 0.06787416 0.66194111 0.33870478 0.08872988 0.97405679]
 [0.00812232 0.06580045 0.73773446 0.28084988 0.03190609 0.9954046 ]
 [0.03165903 0.05734718 0.63479272 0.44711099 0.05321417 0.98430157]
 [0.01088703 0.06134448 0.67510234 0.34646635 0.08596627 0.96785059]
 [0.03205477 0.08792741 0.58487313 0.66871039 0.13704574 0.97979454]
 [0.04333815 0.06355765 0.63525824 0.37443404 0.08720099 0.96966616]
 [0.0198223  0.06625968 0.7453658  0.30021909 0.02384174 0.97791211]
 [0.02412801 0.06698968 0.57917828 0.35034617 0.06208342 0.99780368]
 [0.02532599 0.08149866 0.63034203 0.50568961 0.16771769 0.97648028]
 [0.033418   0.08207265 0.52572446 0.61058777 0.1555315  0.97350193]
 [1.         1.         0.         1.         1.         0.        ]
 [0.01263295 0.06628076 0.50013116 0.55574062 0.04643974 0.98332706]
 [0.01499641 0.06791895 0.55496419 0.49811595 0.07291532 0.98481389]
 [0.02207982 0.08528275 0.62472563 0.57293798 0.14224361 0.9772581 ]
 [0.02793878 0.08073767 0.76755249

In [158]:
y_eval_rf = rf_classifier.predict(X_eval)

print('Test set accuracy for Random Forest: ', metrics.accuracy_score(y_eval, y_eval_rf))

Test set accuracy for Random Forest:  0.38095238095238093


In [159]:
y_pred_svm = svm_classifier.predict(X_eval)

print('Test set accuracy for SVM: ', metrics.accuracy_score(y_eval, y_pred_svm))

Test set accuracy for SVM:  0.38095238095238093
